# Overview

For HashiCorp Vault, Vault Proxy can be used to proxy API calls.  This demo will showcase how Vault Proxy can be used to provide the following capabilities:
- Auth-auth: Vault Proxy authenticates to Vault on behalf of the application.
- Secret caching: Allows caching of static secrets (KVv1 and KVv2) and dynamic secrets. Dynamic secrets will be cached based on the TTL expiry. Static secret caching works with the new Vault event notification system enterprise feature introduced in 1.16. This allows Vault Proxy to refresh the cached entry when write/delete modifications are made to the static secret.

Ref:
- https://developer.hashicorp.com/vault/docs/agent-and-proxy/autoauth
- https://developer.hashicorp.com/vault/docs/agent-and-proxy/proxy/caching

# Setup Vault Enterprise

To run this notebook in VS Code, chose the Jupyter kernel and then Bash

This assumes also your Vault server is already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables

This also assumes you have the Vault CLI installed.

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

In [ ]:
# Optional: You can enable file audit device for more information
docker exec -it vault-enterprise /bin/sh -c "mkdir /var/log/vault.d"
docker exec -it vault-enterprise /bin/sh -c "touch /var/log/vault.d/vault_audit.log"
docker exec -it vault-enterprise /bin/sh -c "chown -R vault:vault /var/log/vault.d"
vault audit enable file file_path=/var/log/vault.d/vault_audit.log

# You can run the following command in the container terminal to follow the logs
# tail -f /var/log/vault.d/vault_audit.log
# Or you can run it from outside on your host machine
# docker exec -it vault-enterprise /bin/sh -c "tail -f /var/log/vault.d/vault_audit.log"
# Use Ctrl + C to break

In [ ]:
# Enable approle Auth, we will use this for the Vault Proxy's authentication
vault auth enable approle

In [ ]:
# Create a KVv2 static secret for the demo, this simulates a credential/secret for an application
vault kv put -mount=secret my-application/my-credentials userid=johndoe password=mypassword

In [ ]:
# Create a policy that will be tied to the approle used by Vault Proxy
vault policy write kv-proxy-policy - << EOF
path "sys/capabilities-self" {
    capabilities = ["update"]
}
path "secret/data/my-application/*" {
  capabilities = ["create", "update", "read", "list", "subscribe"]
  subscribe_event_types = ["*"]
}

path "sys/events/subscribe/*" {
  capabilities = ["read"]
}
EOF

In [ ]:
# Create the approle for the Vault Proxy with the above policy
# Secret ID TTL is currently set to 10m validity.
vault write auth/approle/role/kv-proxy-app-role \
    secret_id_ttl=10m \
    token_num_uses=0 \
    token_ttl=20m \
    token_max_ttl=30m \
    secret_id_num_uses=40 \
    token_policies=kv-proxy-policy,default
# Show settings on app role
vault read auth/approle/role/kv-proxy-app-role

In [ ]:
# Store roleid value for Vault Agent
vault read -field=role_id auth/approle/role/kv-proxy-app-role/role-id > roleid

In [ ]:
# Store secretid value for Vault Agent
vault write -f -field=secret_id auth/approle/role/kv-proxy-app-role/secret-id > secretid

In [ ]:
# Create the Vault Proxy configuration file. The Vault container is exposed via local port 8200.
# For the demo, we will be running Vault Proxy locally in a terminal on port 8100.
tee proxy.hcl <<EOF
pid_file = "./pidfile"

vault {
  address = "http://127.0.0.1:8200"
  retry {
    num_retries = 5
  }
}

auto_auth {
  method {
    type = "approle"
    config = {
      role_id_file_path = "roleid"
      secret_id_file_path = "secretid"
      remove_secret_id_file_after_reading = false
    }
  }
}
cache {
  cache_static_secrets = true
}

api_proxy {
  use_auto_auth_token = "force"
}

listener "tcp" {
    address = "127.0.0.1:8100"
    tls_disable = true
}
log_level="TRACE"
EOF

In [ ]:
# Run the vault proxy in a new terminal window.
# Type ^ + Shift + ` to open a new terminal windows in VS Code
# Copy and paste the follow command to start the Vault proxy
# vault proxy -config=proxy.hcl

# Testing Vault Proxy Auto-Auth and Static Secret Caching

When doing the API call below, you will notice that there is no Vault token required.  This shows the auto-auth feature in action.

In [ ]:
# You can open the docker dashboard and view the Vault container logs side by side as you run this
# Run an API call using curl to simulate the application accessing the secret
# You can see from the terminal window that the Vault Proxy will retrieve the secret from Vault on the first call.
# On subsequent calls, it retreives the secret from the Vault Proxy cache and there is no calls made to Vault subsequently.
curl http://127.0.0.1:8100/v1/secret/data/my-application/my-credentials

## Vault Proxy with Vault event notification system

We will now see the how Vault Proxy is notified of changes on the secret via the Vault event notification system. This is used to refresh the cached entry.

In [ ]:
# Update the password. You can run either of the following commands to alternate updates.
# Note in the terminal that the Vault Proxy will be notified of the write event and it will refresh the cache

In [ ]:
# Update password to 12345678
vault kv put -mount=secret my-application/my-credentials userid=johndoe password=12345678

In [ ]:
# Update password to abcdefgh
vault kv put -mount=secret my-application/my-credentials userid=johndoe password=abcdefgh

In [ ]:
# Verify that the new password is returned from the cache
curl http://127.0.0.1:8100/v1/secret/data/my-application/my-credentials

# Cleanup

In [ ]:
# Terminate the Vault Proxy in the terminal window by using Ctrl + C

In [ ]:
# Cleanup
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

# Remove temp files
# rm cert.json
# rm agent-token
rm roleid
rm secretid
rm proxy.hcl
rm pidfile

# Disable secrets engine and approle auth
vault auth disable approle

# Stop Vault
docker stop vault-enterprise

# Other Useful Commands

In [ ]:
# To subscribe to KVv2 write events
vault events subscribe kv-v2/data-write

In [ ]:
# Retrieve KV secret via Vault CLI
vault kv get -mount=secret my-application/my-credentials